## Wyznaczanie profilu wysokościowego trasy rowerowej

### Autor: Michał Ambroży
### numer: 1, grupa: 1

## Wprowadzenie:
### Celem niniejszego sprawozdania jest wykonanie profilu wysokościowego trasy w systemie wysokości normalnych PL-EVRF2007-NH, opierając się na danych współrzędnych krzywoliniowych oraz wysokości normalnej. Przyjętym modelem wysokości jest aktualny quasigeoida PL-geoid2021. Procedura obejmuje przeliczenie danych wysokości normalnych na wysokości elipsoidalne, a następnie niwelację satelitarną w celu uzyskania wysokości normalnych w układzie PL-EVRF2007.
### 

In [1]:
# Na podstawie danych współrzędnych krzywoliniowych oraz wysokości normalnej trasy w podanym modelu wysokości, wykonaj profil wysokościowy odcinka w systemie wysokości normalnych, w układzie PL-EVRF2007-NH, wykorzystując aktualnie obowiązujący model quasigeoidy:
# PL-geoid2021. Przedstaw przebieg trasy oraz profil na odpowiednich wizualizacjach (wykonaj
# aplikację).
# 1. Pierwszym krokiem jest wykonanie zadania, które nazwać możemy "odwrotną"niwelacją
# satelitarną – na podstawie danych wysokości normalnych, w danym, w zależności od typu
# pliku, systemie wysokości, należy przeliczyć do systemu wysokości elipsoidalnych. Wykorzystane będzie tutaj zatem wzór:
# helip = HN(model) + ζ(model) 
# gdzie:
# • helip – wysokość elipsoidalna,
# • HN(model) – wysokość normalna,
# • ζ(model) – anomalia quasigeoidalna.

import gpxpy
import scipy
from scipy.interpolate import griddata
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import folium as fl

def gpx2df(gpx_file_name, new_file_name = 'route_df2.csv'):
    with open(gpx_file_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    route_info = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time':point.time
                })
    
    route_df = pd.DataFrame(route_info)
    
    save_df(route_df, new_file_name)

def read_route_df(plik):
    df = pd.read_csv(plik)
    return df

def blh2xyz(fi, lam, h) -> np.array:
    a = 6378137.0
    e2 = 0.00669438002290
    N = a/np.sqrt(1-e2*np.sin(fi)**2)
    x = (N+h)*np.cos(fi)*np.cos(lam)
    y = (N+h)*np.cos(fi)*np.sin(lam)
    z = (N*(1-e2)+h)*np.sin(fi)
    xyz = np.array([x,y,z])
    return xyz

def save_df(route_df, new_file_name= 'route_df2.csv'):
    route_df.to_csv(new_file_name, index=False)

def interpolate_geoid(model, phi, lam, tr: scipy.spatial.KDTree = None):
    if tr is None:
        tr = scipy.spatial.KDTree(model[:, :2])
    # find the 4 closest points
    d, idx = tr.query([phi, lam], k=4, workers=-1)
    # calculate weights
    w = 1/d
    # calculate heights
    h = model[idx, 2]
    # calculate weighted average height
    h1 = np.sum(h*w)/np.sum(w)
    return h1

plik = 'test3.gpx'

df = gpx2df(plik)
df = read_route_df('route_df2.csv')

phis = df['latitude'].values
lambdas = df['longitude'].values
dists = [] # dystans od początku trasy w danym punkcie

for i in range(len(phis)-1):
    phi1 = phis[i]
    lam1 = lambdas[i]
    phi2 = phis[i+1]
    lam2 = lambdas[i+1]

    xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
    xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
    dist = np.linalg.norm(xyz1-xyz2)
    dists.append(dist)

# append the last distance
phi1 = phis[-1]
lam1 = lambdas[-1]
phi2 = phis[0]
lam2 = lambdas[0]

xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
dist = np.linalg.norm(xyz1-xyz2)
dists.append(dist)

df['distance'] = np.cumsum(dists)
df['distance'] = df['distance']/1000 # convert to km

kron_model = np.genfromtxt('siatka_gugik-geoid2011-PL-KRON86-NH.txt', skip_header=1)
kron_model = kron_model[~np.isnan(kron_model[:, 2])]

evrf_model = np.genfromtxt('Model_quasi-geoidy-PL-geoid2021-PL-EVRF2007-NH.txt', skip_header=1)
evrf_model = evrf_model[~np.isnan(evrf_model[:, 2])]

# wysokosc elipsoidalna helip = Hn(model geoidy PL-geoid2021) + zeta(model geoidy PL-geoid2021)
helip = []
for lam, phi in zip(lambdas, phis):
    helip.append(interpolate_geoid(evrf_model, phi, lam))
df['helip'] = helip

kron_tr = scipy.spatial.KDTree(kron_model[:, :2])
evrf_tr = scipy.spatial.KDTree(evrf_model[:, :2])
kronHeights = []
evrfHeights = []
for lam, phi in zip(lambdas, phis):
    kronH = interpolate_geoid(kron_model, phi, lam, kron_tr)
    kronHeights.append(kronH)
    evrfH = interpolate_geoid(evrf_model, phi, lam, evrf_tr)
    evrfHeights.append(evrfH)

df['kronH'] = kronHeights
df['evrfH'] = evrfHeights
df['elip'] = df['elevation'] + df['evrfH']
df['HN'] = df['elip'] - df['kronH']
df['diff'] = df['HN'] - df['elevation']

# usun ostatni wiersz df
df = df[:-1]
# wyswietlmy data frame
print(df)

print(f'Wysokość normalna na początku trasy: {df["HN"].iloc[0]}')
print(f'Wysokość normalna na końcu trasy: {df["HN"].iloc[-1]}')

# wyswietlmy roznice wysokosci normalnych w układzie PL-EVRF-2007-NH
print('Rózncia wysokości normalnych w układzie PL-EVRF-2007-NH:') # jednostka: m
print(df['diff'])

      latitude  longitude  elevation  time   distance      helip      kronH  \
0    50.710599  20.893800      245.1   NaN   0.104076  36.304894  36.483629   
1    50.710198  20.895131      246.5   NaN   0.128530  36.298697  36.476922   
2    50.710124  20.895457      246.6   NaN   0.240685  36.298823  36.477418   
3    50.710024  20.897037      246.5   NaN   0.383374  36.297313  36.475911   
4    50.710200  20.899038      247.0   NaN   0.606369  36.295300  36.473895   
..         ...        ...        ...   ...        ...        ...        ...   
114  50.666594  21.082887      227.6   NaN  16.209526  36.041449  36.220211   
115  50.666466  21.083128      228.4   NaN  16.209526  36.041324  36.220053   
116  50.666466  21.083128      228.4   NaN  16.274732  36.041324  36.220053   
117  50.666615  21.084020      227.3   NaN  16.329600  36.040648  36.219285   
118  50.666651  21.084794      230.8   NaN  16.840927  36.040027  36.218577   

         evrfH        elip          HN      diff  


In [2]:
# profil wysokosciowy
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['distance'], y=df['HN'], name='HN'))
# fig.add_trace(go.Scatter(x=df['distance'], y=df['elip'], name='elip'))
fig.add_trace(go.Scatter(x=df['distance'], y=df['elevation'], name='elevation'))
fig.update_layout(title='Profil wysokościowy wybranego szlaku GreenVelo', xaxis_title='Dystans [km]', yaxis_title='Wysokosc [m]')
fig.show()

<!-- Na podstawie danych współrzędnych krzywoliniowych oraz wysokości normalnej trasy w podanym modelu wysokości, wykonaj profil wysokościowy odcinka w systemie wysokości normalnych, w układzie PL-EVRF2007-NH, wykorzystując aktualnie obowiązujący model quasigeoidy:
PL-geoid2021. Przedstaw przebieg trasy oraz profil na odpowiednich wizualizacjach (wykonaj
aplikację). -->


In [3]:
# wysweitl przy pomocy bibioletki foilum najwyzszy punkt trasy na mapie osm
# max_elip = df['elip'].max()
max_elip_idx = df['elip'].idxmax()
max_elip_lat = df['latitude'][max_elip_idx]
max_elip_lon = df['longitude'][max_elip_idx]
print(f'Współrzędne najwyższego punktu trasy: {max_elip_lat}, {max_elip_lon}')

# min_elip = df['elip'].min()
min_elip_idx = df['elip'].idxmin()
min_elip_lat = df['latitude'][min_elip_idx]
min_elip_lon = df['longitude'][min_elip_idx]
print(f'Współrzędne najniższego punktu trasy: {min_elip_lat}, {min_elip_lon}')

# obliczmy srodek trasy
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = fl.Map(location=[center_lat, center_lon], zoom_start=12)
fl.Marker([max_elip_lat, max_elip_lon], popup='Najwyzszy punkt wybranej trasy rowerowej z GreenVelo', tooltip = 'CLick to pop', icon = fl.Icon(color = "green")).add_to(m)
fl.Marker([min_elip_lat, min_elip_lon], popup='Najnizszy punkt wybranej trasy rowerowej z GreenVelo', tooltip = 'CLick to pop', icon = fl.Icon(color = "blue")).add_to(m)
# narysujmy trase na podstawie wpsołrzednych z pliku csv
bicycle_route = df[['latitude', 'longitude']].values.tolist()
fl.PolyLine(bicycle_route, color='red', weight=2.5, opacity=1).add_to(m)
m.save('map.html')
m

Współrzędne najwyższego punktu trasy: 50.705552, 20.931266
Współrzędne najniższego punktu trasy: 50.673976, 21.039663
